In [44]:
import torch
import evaluate
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer, BertConfig, BertTokenizer
# from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict


In [65]:
data = [
    {"text": "什麼是光合作用？", "label": 1},
    {"text": "請解釋一下植物生長的過程。", "label": 1},
    {"text": "植物如何吸收水分和養分？", "label": 1},
    {"text": "哪些植物具有醫療價值？", "label": 1},
    {"text": "如何種植番茄？", "label": 1},
    {"text": "什麼是光合作用的過程？", "label": 1},
    {"text": "植物如何進行生殖繁殖？", "label": 1},
    {"text": "如何識別不同種類的花卉？", "label": 1},
    {"text": "植物如何適應乾旱環境？", "label": 1},
    {"text": "為什麼葉子是綠色的？", "label": 1},
    {"text": "如何種植花園中的蔬菜？", "label": 1},
    {"text": "什麼是有害的植物病蟲害？", "label": 1},
    {"text": "如何護理室內盆栽植物？", "label": 1},
    {"text": "什麼是植物細胞的結構？", "label": 1},
    {"text": "植物是如何進行呼吸作用的？", "label": 1},
    {"text": "如何種植草坪草？", "label": 1},
    {"text": "什麼是盆景藝術？", "label": 1},
    {"text": "植物的種子是如何傳播的？", "label": 1},
    {"text": "如何辨認樹木的種類？", "label": 1},
    {"text": "什麼是植物的生長激素？", "label": 1},
    {"text": "植物如何進行交配和遺傳？", "label": 1},
    {"text": "如何在花園中栽培水果樹？", "label": 1},
    {"text": "什麼是生態園藝學？", "label": 1},
    {"text": "植物是如何感應環境變化的？", "label": 1},
    {"text": "如何保護濒危植物物种？", "label": 1},
    {"text": "什麼是草藥學？", "label": 1},
    {"text": "植物對人类的健康有什麼影響？", "label": 1},
    {"text": "如何種植有機蔬菜？", "label": 1},
    {"text": "什麼是植物營養學？", "label": 1},
    {"text": "植物在生態系統中的角色是什麼？", "label": 1},
    {"text": "如何識別常見的野生花卉？", "label": 1},
    {"text": "什麼是植物的適應性特徵？", "label": 1},
    {"text": "植物在糧食生產中的作用是什麼？", "label": 1},
    {"text": "如何減少植物疾病的傳播？", "label": 1},
    {"text": "什麼是植物生態學？", "label": 1},
    {"text": "什麼是土壤中的微生物生態系統？", "label": 1},
    {"text": "如何識別不同種類的樹木？", "label": 1},
    {"text": "什麼是植物的適應性特徵？", "label": 1},
    {"text": "植物如何在寒冷氣候中生存？", "label": 1},
    {"text": "如何種植有機草坪？", "label": 1},
    {"text": "什麼是生態園藝學？", "label": 1},
    {"text": "植物如何進行光合作用？", "label": 1},
    {"text": "如何減少植物疾病的傳播？", "label": 1},
    {"text": "植物如何吸收水分和養分？", "label": 1},
    {"text": "什麼是植物的生長激素？", "label": 1},
    {"text": "植物如何進行生殖繁殖？", "label": 1},
    {"text": "如何種植蔬菜在室內？", "label": 1},
    {"text": "什麼是森林生態學？", "label": 1},
    {"text": "植物對環境的影響是什麼？", "label": 1},
    {"text": "如何護理家庭室內植物？", "label": 1},
    {"text": "什麼是植物學的研究範疇？", "label": 1},
    {"text": "植物如何適應不同海拔的環境？", "label": 1},
    {"text": "如何區別草本植物的種類？", "label": 1},
    {"text": "什麼是有毒植物的識別方法？", "label": 1},
    {"text": "植物如何進行繁殖的遺傳機制？", "label": 1},
    {"text": "如何在花園中種植水果樹？", "label": 1},
    {"text": "什麼是土壤中的微生物生態系統？", "label": 1},
    {"text": "植物如何參與糧食生產？", "label": 1},
    {"text": "如何種植多年生花卉植物？", "label": 1},
    {"text": "植物如何在生態系統中扮演角色？", "label": 1},
    {"text": "植物的細胞壁是由什麼組成的？", "label": 1},
    {"text": "植物的細胞核具有哪些功能？", "label": 1},
    {"text": "植物的細胞間質有什麼作用？", "label": 1},
    {"text": "植物的維管束組織包括哪兩種主要類型？", "label": 1},
    {"text": "植物的木質部和韌皮部有什麼區別？", "label": 1},
    {"text": "植物的葉片結構包括哪些主要部分？", "label": 1},
    {"text": "植物的葉片上的氣孔有什麼功能？", "label": 1},
    {"text": "植物的根系結構包括哪些主要部分？", "label": 1},
    {"text": "植物的根體和根冠有什麼不同？", "label": 1},
    {"text": "植物如何吸收水分和養分通過根系？", "label": 1},
    {"text": "植物的莖結構包括哪些主要部分？", "label": 1},
    {"text": "植物的莖的主要功能是什麼？", "label": 1},
    {"text": "植物的花朵的不同部分包括什麼？", "label": 1},
    {"text": "植物的花朵如何進行有性生殖？", "label": 1},
    {"text": "植物的果實是如何形成的？", "label": 1},
    {"text": "植物的種子結構包括哪些主要部分？", "label": 1},
    {"text": "植物的種子如何傳播？", "label": 1},
    {"text": "植物的花粉如何傳播？", "label": 1},
    {"text": "植物的生殖器官是什麼？", "label": 1},
    {"text": "植物的有性生殖和無性生殖有什麼區別？", "label": 1},
    {"text": "植物如何進行交叉授粉？", "label": 1},
    {"text": "植物的雄蕊和雌蕊分別是什麼？", "label": 1},
    {"text": "植物的光合作用是在哪個部分進行的？", "label": 1},
    {"text": "植物的葉綠素在光合作用中的作用是什麼？", "label": 1},
    {"text": "植物如何進行光合作用的碳固定？", "label": 1},
    {"text": "植物如何調節水分蒸發通過葉子？", "label": 1},
    {"text": "植物的植物生長激素有哪些種類？", "label": 1},
    {"text": "植物的生長激素對植物生長有什麼影響？", "label": 1},
    {"text": "植物的葉片結構如何適應光照變化？", "label": 1},
    {"text": "植物的葉片上的葉脈有什麼作用？", "label": 1},
    {"text": "植物的葉片顏色和光合作用有關嗎？", "label": 1},
    {"text": "植物如何進行水分運輸通過植物體？", "label": 1},
    {"text": "植物的細胞間質是如何幫助光合作用的？", "label": 1},
    {"text": "植物的根系結構如何適應不同土壤類型？", "label": 1},
    {"text": "植物的木質部如何提供支持和運輸？", "label": 1},
    {"text": "植物的果實結構如何吸引動物傳播種子？", "label": 1},
    {"text": "植物如何抵抗害蟲和病原體？", "label": 1},
    {"text": "植物的適應性特徵如何幫助其在特定環境中生存？", "label": 1},
    {"text": "植物如何感應光照的方向和強度？", "label": 1},
    {"text": "植物的葉片表面結構如何減少水分蒸發？", "label": 1},
    {"text": "植物的種子如何適應不同傳播方式？", "label": 1},
    {"text": "植物的莖的不同結構對適應性有什麼影響？", "label": 1},
    {"text": "植物的生長體的功能是什麼？", "label": 1},
    {"text": "植物的花朵的形狀和顏色如何吸引授粉者？", "label": 1},
    {"text": "植物如何感應光照和重力方向？", "label": 1},
    {"text": "植物的葉片內部結構如何最大程度地提高光合作用？", "label": 1},
    {"text": "植物的根系如何吸收特定養分？", "label": 1},
    {"text": "植物的莖結構如何適應高風或高溼度環境？", "label": 1},
    {"text": "植物如何感應日夜變化而進行開花？", "label": 1},
    {"text": "植物的葉片在夜晚做什麼生理活動？", "label": 1},
    {"text": "植物如何感應氣溫變化並調整生長速率？", "label": 1},
    {"text": "植物在幾種不同的光照條件下表現不同的行為？", "label": 1},
    {"text": "植物如何感應周圍植物和其他生物的存在？", "label": 1},
    {"text": "植物如何感應乾旱條件並調整水分利用效率？", "label": 1},
    {"text": "植物的開花時間如何受到光照和溫度的影響？", "label": 1},
    {"text": "植物的光合作用率在不同時段有何變化？", "label": 1},
    {"text": "植物如何調整葉片的角度以最大程度地吸收陽光？", "label": 1},
    {"text": "植物如何感應風的存在並調整莖的生長方式？", "label": 1},
    {"text": "植物如何感應重力方向並調整根的生長？", "label": 1},
    {"text": "植物在光照不足時如何進行呼吸作用？", "label": 1},
    {"text": "植物如何感應病原體入侵並啟動防禦機制？", "label": 1},
    {"text": "植物如何通過化學物質進行互相溝通？", "label": 1},
    {"text": "植物如何感應土壤養分濃度並調整根的生長？", "label": 1},
    {"text": "植物如何在水中環境中進行生長和繁殖？", "label": 1},
    {"text": "植物如何感應光照方向並調整植物的方向生長？", "label": 1},
    {"text": "植物如何感應氣體濃度變化並調整呼吸作用速率？", "label": 1},
    {"text": "植物如何通過花粉吸引授粉者進行有性生殖？", "label": 1},
    {"text": "植物如何感應水分濃度並調整氣孔開合？", "label": 1},
    {"text": "植物的根系如何感應土壤中的毒物並進行解毒？", "label": 1},
    {"text": "植物如何感應病蟲害入侵並啟動防禦反應？", "label": 1},
    {"text": "植物如何感應植物周圍的空間競爭並調整生長方式？", "label": 1},
    {"text": "植物如何通過發送化學訊號吸引有益生物？", "label": 1},
    {"text": "植物如何通過花香吸引昆蟲授粉者？", "label": 1},
    {"text": "植物如何感應週遭溫度變化並調整開花時機？", "label": 1},
    {"text": "植物如何感應光線強度和光譜並調整生長方式？", "label": 1},
    {"text": "植物如何感應土壤中的微生物生態系統並進行協作？", "label": 1},
    {"text": "植物如何感應土壤pH值並調整養分吸收？", "label": 1},
    {"text": "植物如何感應氣體濃度變化並調整氣孔開合？", "label": 1},
    {"text": "植物如何感應光照週期並調整開花和結實？", "label": 1},
    {"text": "植物如何感應風的強度和頻率並調整莖的生長？", "label": 1},
    {"text": "植物如何感應土壤中的化學養分分佈並調整根的生長？", "label": 1},
    {"text": "植物如何感應外部機械刺激並進行生長和防禦反應？", "label": 1},
    {"text": "植物如何感應植物之間的競爭壓力並調整生長速率？", "label": 1},
    {"text": "植物如何感應環境中的溫度變化並調整生長方式？", "label": 1},
    {"text": "植物如何感應土壤中的水分含量並調整根的生長？", "label": 1},
    {"text": "植物如何感應植物內部的養分分配並調整生長？", "label": 1},
    {"text": "植物如何感應化學物質信號並進行互相合作？", "label": 1},
    {"text": "植物如何感應外部機械損傷並進行自我修復？", "label": 1},
    {"text": "植物如何感應土壤中的水分濃度並調整根的生長速率？", "label": 1},
    {"text": "植物如何感應風的振動並調整生長方式？", "label": 1},
    {"text": "植物如何感應光照和風的方向並調整植物的方向生長？", "label": 1},
    {"text": "什麼是被子植物，它們的特徵是什麼？", "label": 1},
    {"text": "被子植物和裸子植物有什麼不同之處？", "label": 1},
    {"text": "植物的分類系統中包括哪些主要分類單位？", "label": 1},
    {"text": "草本植物和木本植物之間有什麼區別？", "label": 1},
    {"text": "什麼是蕨類植物，它們的生活週期如何？", "label": 1},
    {"text": "依據種子數量，植物可以分為哪兩大類？", "label": 1},
    {"text": "什麼是紅藻，它們屬於哪一個植物分類？", "label": 1},
    {"text": "什麼是苔藓植物，它們的特徵是什麼？", "label": 1},
    {"text": "綠藻和植物之間有什麼親緣關係？", "label": 1},
    {"text": "什麼是離強花，它們屬於哪一個植物分類？", "label": 1},
    {"text": "如何區分單子葉植物和雙子葉植物？", "label": 1},
    {"text": "什麼是顯花植物，它們的特點是什麼？", "label": 1},
    {"text": "被子植物的種子如何發育？", "label": 1},
    {"text": "什麼是木質植物，它們的特徵和生活週期如何？", "label": 1},
    {"text": "草本植物和灌木之間有什麼區別？", "label": 1},
    {"text": "蕨類植物的孢子如何形成和繁殖？", "label": 1},
    {"text": "被子植物的種子分為哪些部分，各自的功能是什麼？", "label": 1},
    {"text": "苔藓植物的生長環境和生活史是什麼？", "label": 1},
    {"text": "植物分類中的家族和屬是如何定義的？", "label": 1},
    {"text": "什麼是角藻，它們的特點和生態如何？", "label": 1},
    {"text": "如何區分針葉樹和闊葉樹？", "label": 1},
    {"text": "什麼是單子葉植物，它們的特徵和分布如何？", "label": 1},
    {"text": "雙子葉植物的花朵結構和特點是什麼？", "label": 1},
    {"text": "被子植物的花朵如何進行受精和種子形成？", "label": 1},
    {"text": "草本植物的生活週期和繁殖方式如何？", "label": 1},
    {"text": "什麼是藻類，它們的分類和生活環境如何？", "label": 1},
    {"text": "被子植物的根系結構和功能是什麼？", "label": 1},
    {"text": "什麼是木本植物的次生生長，它們如何增加植株體積？", "label": 1},
    {"text": "蕨類植物的胚珠如何發育成成熟植株？", "label": 1},
    {"text": "植物的分類和命名有何主要規則和系統？", "label": 1},
    {"text": "什麼是藻類的色素體，它們對光合作用有何影響？", "label": 1},
    {"text": "被子植物的莖結構和功能是什麼？", "label": 1},
    {"text": "什麼是草本植物的開花和結果過程，它們的生育週期是什麼？", "label": 1},
    {"text": "蕨類植物的孢子如何傳播，以實現繁殖？", "label": 1},
    {"text": "植物的分類中有哪些主要類群，它們之間的演化關係如何？", "label": 1},
    {"text": "什麼是苔藓植物的莖和葉，它們如何進行生長和營養吸收？", "label": 1},
    {"text": "藻類的細胞結構和功能是什麼，它們如何適應不同的水域環境？", "label": 1},
    {"text": "被子植物的花朵和種子如何進行傳播，以實現繁殖？", "label": 1},
    {"text": "什麼是木本植物的生長環境和生活史，它們如何適應不同的生態系統？", "label": 1},
    {"text": "草本植物的根系結構和功能是什麼，它們如何吸收水分和養分？", "label": 1},
    {"text": "蕨類植物的孢子如何發育成新的植株，它們的胚珠如何形成？", "label": 1},
    {"text": "植物的分類中有哪些特殊植物群，例如食肉植物和空氣植物？", "label": 1},
    {"text": "什麼是藻類的生殖器官，它們如何進行生殖？", "label": 1},
    {"text": "被子植物的莖和葉的特徵是什麼，它們如何進行光合作用？", "label": 1},
    {"text": "苔藓植物如何在乾旱環境中生存，它們的葉片結構和蒸散作用如何？", "label": 1},
    {"text": "藻類的多樣性如何，它們的細胞結構和色素體有何不同？", "label": 1},
    {"text": "木本植物的生殖器官如何進行受精，它們的果實如何形成？", "label": 1},
    {"text": "什麼是草本植物的花朵結構，它們如何進行授粉和結果？", "label": 1},
    {"text": "被子植物的種子如何進行傳播，它們的種皮和胚珠結構如何？", "label": 1},
    {"text": "苔藓植物如何對水分敏感，它們的生長方式和循環有何特點？", "label": 1},
    {"text": "藻類如何在水域生態中發揮重要角色，它們的細胞結構和生殖方式如何？", "label": 1},
    {"text": "如何在家裡節約能源？", "label": 0},
    {"text": "為什麼應該關心塑料垃圾對海洋的影響？", "label": 0},
    {"text": "最新的手機應用程式有哪些有趣的功能？", "label": 0},
    {"text": "怎麼應對城市交通擁擠問題？", "label": 0},
    {"text": "什麼是當前的科技趨勢？", "label": 0},
    {"text": "如何在互聯網上保護你的隱私？", "label": 0},
    {"text": "為什麼太陽能電池是綠能的未來？", "label": 0},
    {"text": "如何提高城市空氣品質？", "label": 0},
    {"text": "什麼是大數據，如何應用於我們的生活？", "label": 0},
    {"text": "怎麼保護野生動物的棲息地？", "label": 0},
    {"text": "為什麼應該關心氣候變化和碳排放？", "label": 0},
    {"text": "最新的科技如何影響我們的生活？", "label": 0},
    {"text": "如何應用虛擬現實技術於醫療保健？", "label": 0},
    {"text": "什麼是自動駕駛汽車技術的現況？", "label": 0},
    {"text": "怎麼在網絡上保護你的個人資料？", "label": 0},
    {"text": "什麼是太空探索的最新進展？", "label": 0},
    {"text": "如何處理核廢料？", "label": 0},
    {"text": "如何設計一個出色的網頁應用程式？", "label": 0},
    {"text": "怎樣減少家庭能源消耗？", "label": 0},
    {"text": "簡單的辦法減少塑料使用？", "label": 0},
    {"text": "最佳的健身方法是什麼？", "label": 0},
    {"text": "如何應對社交媒體的信息過載？", "label": 0},
    {"text": "如何在家裡種植有機蔬菜？", "label": 0},
    {"text": "為什麼要支持性別平等？", "label": 0},
    {"text": "未來的環境永續發展有什麼挑戰？", "label": 0},
    {"text": "什麼是人工智慧？", "label": 0},
    {"text": "怎麼減少塑料污染？", "label": 0},
    {"text": "請解釋一下太陽系的形成。", "label": 0},
    {"text": "如何改善交通擁擠問題？", "label": 0},
    {"text": "什麼是最新的科技趨勢？", "label": 0},
    {"text": "什麼是太陽能電池的工作原理？", "label": 0},
    {"text": "如何改進風能發電的效率？", "label": 0},
    {"text": "什麼是區塊鏈技術？", "label": 0},
    {"text": "如何減少塑料垃圾對海洋生態的影響？", "label": 0},
    {"text": "如何在城市規劃中考慮可持續性？", "label": 0},
    {"text": "人工智慧如何用於醫學診斷？", "label": 0},
    {"text": "什麼是大數據分析？", "label": 0},
    {"text": "如何保護野生動物棲息地？", "label": 0},
    {"text": "如何進行環境保護教育？", "label": 0},
    {"text": "什麼是人類工機互動技術？", "label": 0},
    {"text": "如何改善城市空氣質量？", "label": 0},
    {"text": "什麼是量子計算？", "label": 0},
    {"text": "如何在網絡安全方面保護個人隱私？", "label": 0},
    {"text": "什麼是虛擬現實技術？", "label": 0},
    {"text": "如何設計可持續的城市交通系統？", "label": 0},
    {"text": "什麼是生物多樣性保護？", "label": 0},
    {"text": "如何降低碳排放並減緩氣候變化？", "label": 0},
    {"text": "什麼是機器學習的未來趨勢？", "label": 0},
    {"text": "如何在醫療保健中應用虛擬現實？", "label": 0},
    {"text": "什麼是自動駕駛汽車技術？", "label": 0},
    {"text": "如何在互聯網上保護個人數據？", "label": 0},
    {"text": "什麼是太空探索的最新進展？", "label": 0},
    {"text": "如何處理核廢料？", "label": 0},
    {"text": "如何設計一個網頁應用程式？", "label": 0},
    {"text": "怎樣減少家庭能源消耗？", "label": 0},
    {"text": "請解釋一下物理學中的牛頓定律。", "label": 0},
    {"text": "哪些方法可以提高學習效率？", "label": 0},
    {"text": "什麼是太陽能電池，它是如何工作的？", "label": 0},
    {"text": "如何製作一道美味的巧克力蛋糕？", "label": 0},
    {"text": "最新的手機應用程式有哪些值得關注的功能？", "label": 0},
    {"text": "什麼是歷史學中的文化交流？", "label": 0},
    {"text": "古代埃及的金字塔是如何建造的？", "label": 0},
    {"text": "誰是莎士比亞，他的作品有哪些？", "label": 0},
    {"text": "什麼是地理學中的環境可持續性？", "label": 0},
    {"text": "如何解釋物理學中的相對論？", "label": 0},
    {"text": "生物學中的遺傳學有哪些基本原理？", "label": 0},
    {"text": "什麼是心理學中的行為主義？", "label": 0},
    {"text": "藝術史上的文藝復興運動是什麼，它如何改變藝術领域？", "label": 0},
    {"text": "化學中的元素周期表如何組織各種元素？", "label": 0},
    {"text": "什麼是政治學中的民主和獨裁主義？", "label": 0},
    {"text": "數學中的勾股定理是什麼，如何應用？", "label": 0},
    {"text": "地球科學中的板塊運動是什麼，它如何造成地震和火山活動？", "label": 0},
    {"text": "什麼是文學中的象徵主義運動，有哪些代表作品？", "label": 0},
    {"text": "社會學中的社會階層是如何形成的，它對社會有哪些影響？", "label": 0},
    {"text": "音樂史上的古典音樂時期有哪些著名作曲家和作品？", "label": 0},
    {"text": "什麼是經濟學中的市場競爭，它如何影響價格和供求？", "label": 0},
    {"text": "物理學中的電磁學涉及哪些主題，如電場和磁場？", "label": 0},
    {"text": "心理學中的心理疾病診斷和治療方法有哪些？", "label": 0},
    {"text": "什麼是藝術中的印象派運動，有哪些知名畫家和作品？", "label": 0},
    {"text": "地理學中的人口結構如何影響國家和區域發展？", "label": 0},
    {"text": "化學中的有機化學是什麼，它研究有機分子的性質和反應？", "label": 0},
    {"text": "政治學中的國際關係理論包括現實主義和自由主義，它們有什麼不同？", "label": 0},
    {"text": "什麼是環境科學中的生態系統，它如何保護生態平衡？", "label": 0},
    {"text": "法律學中的刑法和民法有哪些區別，它們如何影響社會和個人？", "label": 0},
    {"text": "醫學中的疾病預防和治療方法有哪些，如疫苗和藥物？", "label": 0},
    {"text": "數學中的微積分是什麼，它如何應用於物理和工程學？", "label": 0},
    {"text": "體育科學中的運動生理學研究運動對人體的影響，如肌肉和心臟功能？", "label": 0},
    {"text": "什麼是宗教學中的宗教多元主義，它探討不同宗教信仰的關係？", "label": 0},
    {"text": "社會學中的社會運動和抗議如何影響政策和社會變革？", "label": 0},
    {"text": "什麼是文化研究，它探討文化如何影響語言、藝術和價值觀？", "label": 0},
    {"text": "電子工程中的電路設計和電子元件有哪些關鍵原則？", "label": 0},
    {"text": "飲食營養學中的基本營養需求和飲食計劃如何促進健康？", "label": 0},
    {"text": "什麼是環境倫理學，它探討倫理問題在環境保護和永續發展方面的應用？", "label": 0},
    {"text": "經濟學中的通膨和失業如何影響國家經濟，並如何應對？", "label": 0},
    {"text": "生命科學中的基因編輯技術如CRISPR有哪些潛在應用，以及倫理和法律問題？", "label": 0},
    {"text": "什麼是藝術治療，它如何應用藝術創作來促進心理和情感健康？", "label": 0},
    {"text": "心理學中的行為主義和認知心理學的理論有哪些差異，如何解釋學習和思維？", "label": 0},
    {"text": "數學中的統計學如何用於數據分析，並對科學研究和決策產生影響？", "label": 0},
    {"text": "藝術中的現代主義和後現代主義運動有哪些特點，它們如何影響藝術作品？", "label": 0},
    {"text": "地理學中的地球系統科學探討地球的自然和人為系統，如氣候和生態系統之間的交互作用？", "label": 0},
    {"text": "法律學中的國際法和國內法的區別是什麼，如何維護國家和國際秩序？", "label": 0},
    {"text": "音樂學中的音樂理論和和聲分析如何解釋音樂的結構和表現？", "label": 0},
    {"text": "政治學中的政府形式如民主、專制、君主制和議會制度有哪些特點，它們如何運作？", "label": 0},
    {"text": "生物學中的生態學研究生物和環境之間的相互關係，如食物鏈和生態系統平衡？", "label": 0},
    {"text": "什麼是宗教研究，它探討不同宗教的信仰、實踐和宗教歷史？", "label": 0},
    {"text": "什麼是歷史學中的文化交流？", "label": 0},
    {"text": "古代埃及的金字塔是如何建造的？", "label": 0},
    {"text": "誰是莎士比亞，他的作品有哪些？", "label": 0},
    {"text": "什麼是地理學中的環境可持續性？", "label": 0},
    {"text": "如何解釋物理學中的相對論？", "label": 0},
    {"text": "生物學中的遺傳學有哪些基本原理？", "label": 0},
    {"text": "什麼是心理學中的行為主義？", "label": 0},
    {"text": "藝術史上的文藝復興運動是什麼，它如何改變藝術领域？", "label": 0},
    {"text": "化學中的元素周期表如何組織各種元素？", "label": 0},
    {"text": "什麼是政治學中的民主和獨裁主義？", "label": 0},
    {"text": "數學中的勾股定理是什麼，如何應用？", "label": 0},
    {"text": "地球科學中的板塊運動是什麼，它如何造成地震和火山活動？", "label": 0},
    {"text": "什麼是文學中的象徵主義運動，有哪些代表作品？", "label": 0},
    {"text": "社會學中的社會階層是如何形成的，它對社會有哪些影響？", "label": 0},
    {"text": "音樂史上的古典音樂時期有哪些著名作曲家和作品？", "label": 0},
    {"text": "什麼是經濟學中的市場競爭，它如何影響價格和供求？", "label": 0},
    {"text": "物理學中的電磁學涉及哪些主題，如電場和磁場？", "label": 0},
    {"text": "心理學中的心理疾病診斷和治療方法有哪些？", "label": 0},
    {"text": "什麼是藝術中的印象派運動，有哪些知名畫家和作品？", "label": 0},
    {"text": "地理學中的人口結構如何影響國家和區域發展？", "label": 0},
    {"text": "化學中的有機化學是什麼，它研究有機分子的性質和反應？", "label": 0},
    {"text": "政治學中的國際關係理論包括現實主義和自由主義，它們有什麼不同？", "label": 0},
    {"text": "什麼是環境科學中的生態系統，它如何保護生態平衡？", "label": 0},
    {"text": "法律學中的刑法和民法有哪些區別，它們如何影響社會和個人？", "label": 0},
    {"text": "醫學中的疾病預防和治療方法有哪些，如疫苗和藥物？", "label": 0},
    {"text": "數學中的微積分是什麼，它如何應用於物理和工程學？", "label": 0},
    {"text": "體育科學中的運動生理學研究運動對人體的影響，如肌肉和心臟功能？", "label": 0},
    {"text": "什麼是宗教學中的宗教多元主義，它探討不同宗教信仰的關係？", "label": 0},
    {"text": "社會學中的社會運動和抗議如何影響政策和社會變革？", "label": 0},
    {"text": "什麼是文化研究，它探討文化如何影響語言、藝術和價值觀？", "label": 0},
    {"text": "電子工程中的電路設計和電子元件有哪些關鍵原則？", "label": 0},
    {"text": "飲食營養學中的基本營養需求和飲食計劃如何促進健康？", "label": 0},
    {"text": "什麼是犯罪學，它研究犯罪的成因和犯罪防治？", "label": 0},
    {"text": "教育學中的教育心理學探討學習和教學的心理過程，如學生動機和記憶？", "label": 0},
    {"text": "什麼是音樂學中的音樂教育，它如何促進音樂素養和表演能力？", "label": 0},
    {"text": "科技與社會學中的數字鴿洞理論如何解釋技術和社會互動？", "label": 0},
    {"text": "體育學中的運動訓練方法如何提高運動表現和預防傷害？", "label": 0},
    {"text": "什麼是文學研究中的文學理論，如結構主義和後現代主義？", "label": 0},
    {"text": "經濟學中的國際貿易和關稅政策如何影響國家經濟？", "label": 0},
    {"text": "醫學倫理學探討醫療決策和生命倫理問題，如生殖技術和安樂死？", "label": 0},
    {"text": "什麼是環境科學中的氣候變化，它如何影響地球的生態系統？", "label": 0},
    {"text": "心理學中的社會心理學研究人類行為和互動，如群體行為和決策？", "label": 0},
    {"text": "法律倫理學探討法律和倫理的關係，如律師道德和司法公正？", "label": 0},
    {"text": "什麼是健康管理，它如何促進醫療機構和病人的管理？", "label": 0},
    {"text": "食品科學中的食品加工技術如何影響食品品質和安全？", "label": 0},
    {"text": "什麼是哲學中的倫理學，它探討道德價值觀和道德決策？", "label": 0},
    {"text": "體育醫學中的運動傷害診斷和治療方法有哪些，如跌打和扭傷？", "label": 0},
    {"text": "什麼是媒體研究，它探討媒體和文化之間的關係，如新聞和娛樂？", "label": 0},
    {"text": "軟體工程中的軟體開發方法如敏捷開發和瀑布模型有哪些差異？", "label": 0},
    {"text": "什麼是教育政策，它如何影響學校和教育體系的運作？", "label": 0},
    {"text": "室內設計中的空間規劃和家具設計如何創造功能性和美觀的室內環境？", "label": 0},
    {"text": "什麼是圖書館學，它探討圖書館和信息科學的管理和服務？", "label": 0},
    {"text": "市場營銷中的數位營銷策略如何利用網絡和社交媒體？", "label": 0},
    {"text": "什麼是數學中的群論，它研究代數結構和對稱性？", "label": 0},
    {"text": "酒店管理中的客房運營和客戶服務如何確保良好的客人體驗？", "label": 0},
    {"text": "如何在旅遊中安排預算旅行？", "label": 0},
    {"text": "最佳的時間管理技巧是什麼？", "label": 0},
    {"text": "怎樣維護健康的飲食習慣？", "label": 0},
    {"text": "如何減少家庭浪費和回收垃圾？", "label": 0},
    {"text": "探討現代藝術和藝術家的作品。", "label": 0},
    {"text": "如何安排一個成功的社交活動或派對？", "label": 0},
    {"text": "探討流行文化和電影中的時尚趨勢。", "label": 0},
    {"text": "如何選擇最適合你的旅遊目的地？", "label": 0},
    {"text": "健康生活方式中的心理健康技巧和壓力管理。", "label": 0},
    {"text": "如何設計個性化的家居裝飾和佈置？", "label": 0},
    {"text": "探討當今的音樂流行趨勢和藝人。", "label": 0},
    {"text": "如何有效處理家庭和工作的平衡？", "label": 0},
    {"text": "介紹當前的流行電視劇和影集。", "label": 0},
    {"text": "如何在社交媒體上建立個人品牌和在線存在？", "label": 0},
    {"text": "生活在現代數碼時代中的網絡安全和個人隱私。", "label": 0},
    {"text": "如何培養一個成功的家庭菜園或室內植物？", "label": 0},
    {"text": "教育小孩儿培養閱讀和學習興趣的方法。", "label": 0},
    {"text": "探討美容和護膚的最新趨勢和產品。", "label": 0},
    {"text": "如何在網上購物時找到最佳的優惠和促銷？", "label": 0},
    {"text": "健康生活方式中的運動和體能鍛煉建議。", "label": 0},
    {"text": "探討科技和社交媒體的成癮現象和減少屏幕時間。", "label": 0},
    {"text": "如何應對家庭和夫妻關係中的挑戰和衝突？", "label": 0},
    {"text": "介紹最受歡迎的手機應用程式和其功能。", "label": 0},
    {"text": "如何規劃一次成功的遠足或露營之旅？", "label": 0},
    {"text": "探討飲食和烹飪中的健康飲食和食譜。", "label": 0},
    {"text": "簡單的家庭DIY項目和室內裝修建議。", "label": 0},
    {"text": "如何在每日生活中實踐環保和減少碳足跡？", "label": 0},
    {"text": "生活方式中的自我關愛和心靈成長建議。", "label": 0},
    {"text": "探討健身和運動訓練中的最新趨勢和方法。", "label": 0},
    {"text": "如何管理個人財務和制定預算計劃？", "label": 0},
    {"text": "家庭和寵物護理中的照顧和飼養建議。", "label": 0},
    {"text": "探討旅遊和冒險中的最佳目的地和體驗。", "label": 0},
    {"text": "如何維護和修復家用電器和設備？", "label": 0},
    {"text": "生活中的社交技巧和人際關係建議。", "label": 0},
    {"text": "探討美食和餐廳評論中的最佳餐廳和烹飪技巧。", "label": 0},
    {"text": "如何選擇最適合你的寵物和寵物護理技巧？", "label": 0},
    {"text": "健康生活方式中的心理健康和壓力管理建議。", "label": 0},
    {"text": "探討電影和電視劇中的最新娛樂和觀影建議。", "label": 0},
    {"text": "如何建立健康的家庭飲食和烹飪技巧？", "label": 0},
    {"text": "生活中的時尚和穿著建議，如造型和購物技巧。", "label": 0},
    {"text": "探討寵物和動物護理中的最新趨勢和飼養建議。", "label": 0},
    {"text": "如何選擇最適合你的旅遊和度假計劃？", "label": 0},
    {"text": "健康生活方式中的飲食和烹飪建議，如食譜和營養信息。", "label": 0},
    {"text": "探討科技和數位世界中的最新應用和數碼技巧。", "label": 0},
]


train_data, valid_data = train_test_split(data, test_size=0.2, random_state=42)

train_data_dict = {
    "text": [item["text"] for item in train_data],
    "label": [item["label"] for item in train_data],
}

valid_data_dict = {
    "text": [item["text"] for item in valid_data],
    "label": [item["label"] for item in valid_data],
}

train_dataset = Dataset.from_dict(train_data_dict)
valid_dataset = Dataset.from_dict(valid_data_dict)

dataset = DatasetDict({'train': train_dataset, 'valid': valid_dataset})
print(dataset)

model_name = "iioSnail/bert-base-chinese-word-classifier"

# 1. 創建原始模型
origin_model = BertForSequenceClassification.from_pretrained(model_name)

# 2. 創建新模型配置，設置為只有2個標籤
new_model_config = BertConfig.from_pretrained(model_name)
new_model_config.num_labels = 2

# 3. 創建新的模型，只有2種label
new_model = BertForSequenceClassification(new_model_config)

# 4. 更新新模型的label映射
new_model.config.id2label = {0: "非植物", 1: "植物"}
new_model.config.label2id = {"非植物": 0, "植物": 1}

# 5. 複製原始模型的權重
new_model.bert.load_state_dict(origin_model.bert.state_dict())


#5. 儲存新模型以供後續使用
new_model.save_pretrained('new_model_directory')
# tokenizer.save_pretrained('new_model_directory')

model = BertForSequenceClassification.from_pretrained('new_model_directory', )
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_data = dataset.map(preprocess_function, batched=True)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir="intent_model_3",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=new_model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['valid'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 324
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 82
    })
})


Map:   0%|          | 0/324 [00:00<?, ? examples/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

  0%|          | 0/105 [00:00<?, ?it/s]

ValueError: Target size (torch.Size([16])) must be the same as input size (torch.Size([16, 2]))